In [22]:
# !pip install yahooquery

from yahooquery import Ticker
import pandas as pd
import plotly.graph_objects as go
import numpy as np
import statsmodels.api as sm
from scipy.stats import linregress   

Можно пропробовать сделать так, чтобы вместо input, пользователь вводил имя тикера в чат телеграм боту, после чего программа бы запускалась

In [45]:
#In the following inputs user should insert tickers of variables from yahoofinance.com. 
#If using testing example, insert nothing
t_main=input('Insert ticker of influenced variable: ')
t_inf1=input('Insert first influencing ticker: ')
t_inf2=input('Insert second influencing ticker: ')
if t_main=='' or t_inf1=='' or t_inf2=='':
    t_main='RUBUSD=X'
    t_inf1='NG=F'
    t_inf2='CL=F'
    
tick_m=Ticker(t_main)
tick_1=Ticker(t_inf1)
tick_2=Ticker(t_inf2)


#import data
data_m=tick_m.history(start='2002-01-01', interval = '1d')
data_1=tick_1.history(start='2002-01-01', interval = '1d') #'gas data'
data_2=tick_2.history(start='2002-01-01', interval = '1d') #'oil data'

#Check that data installed properly
if len(data_m)==1 or len(data_1)==1 or len(data_2)==1:
    print('Not properly inserted tickers, might lead to further mistakes')
data_m    

Insert ticker of influenced variable: 
Insert first influencing ticker: 
Insert second influencing ticker: 


open      high       low  volume     close  adjclose
symbol   date                                                                
RUBUSD=X 2003-12-01  0.033636  0.033681  0.033636     0.0  0.033670  0.033670
         2003-12-02  0.033681  0.033681  0.033670     0.0  0.033681  0.033681
         2003-12-03  0.033681  0.033681  0.033681     0.0  0.033681  0.033681
         2003-12-04  0.033693  0.033693  0.033693     0.0  0.033693  0.033693
         2003-12-05  0.033750  0.033750  0.033750     0.0  0.033761  0.033761
...                       ...       ...       ...     ...       ...       ...
         2022-05-10  0.014318  0.014660  0.014277     0.0  0.014318  0.014318
         2022-05-11  0.014318  0.015131  0.014318     0.0  0.014318  0.014318
         2022-05-12  0.014989  0.015695  0.014904     0.0  0.014989  0.014989
         2022-05-13  0.015305  0.015924  0.015131     0.0  0.015363  0.015363
         2022-05-14  0.015504  0.015504  0.015504     0.0  0.015504  0.015504

[4678 rows x 6 columns]

Для наглядности здесь можно сделать два графика: 1)Linechart(Scatterplot с соединениями) по трем таблицам , используя Dropbox для сортировки по виду данных(Close или Open и тд.) 2) 3D-график с осью z(вертикальной)-курс рубля, используя Slider для фильтра по годам.

Надо будет сделать выводы по полученным данным

Set common dates for propriate further tests

In [46]:
dates_m = data_m.index.get_level_values(level=1)
dates1 = data_1.index.get_level_values(level=1)
dates2 = data_2.index.get_level_values(level=1)
merged_dates = list(set(dates1) & set(dates2) & set(dates_m))

In [63]:
data_m_=data_m.loc['RUBUSD=X'].loc[merged_dates]
data_1_=data_1.loc['NG=F'].loc[merged_dates]
data_2_=data_2.loc['CL=F'].loc[merged_dates]


# Use regression to test the data


In [64]:
#get data for close column
y_=list(data_m_['close'])
x1=list(data_1_['close'])
x2=list(data_2_['close'])
#remove outliers
for i in range(len(y_)):
    if y_[i]>1:
        y_.pop(i)
        break
plot = go.Figure()

graph = go.Scatter(x = x1, y = y_, mode = 'markers')
plot.update_layout(title='Ruble/gas dependence',xaxis_title='Gas',yaxis_title='Ruble/Dollar exchange rate')
plot.add_trace(graph)

In [65]:
plot2=go.Figure()
#remove outliers
for i in range(len(x2)):
    if x2[i]<0:
        x2.pop(i)
        break
graph = go.Scatter(x = x2, y = y_, mode = 'markers')
plot2.update_layout(title='Ruble/oil dependence',xaxis_title='Oil',yaxis_title='Ruble/Dollar exchange rate')

plot2.add_trace(graph)

Вероятно, можно представить графики более доступно и добавить что-то(если есть желание)

Analytical test of ruble/gas dependance

In [66]:
#use regression analysis on ruble/gas prices relationship
x1_ = sm.add_constant(list(x1))
model = sm.OLS(y_,x1_[:len(y_)])
res1 = model.fit()
res1.summary()


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.425
Model:                            OLS   Adj. R-squared:                  0.425
Method:                 Least Squares   F-statistic:                     3330.
Date:                Sun, 15 May 2022   Prob (F-statistic):               0.00
Time:                        13:54:08   Log-Likelihood:                 15691.
No. Observations:                4506   AIC:                        -3.138e+04
Df Residuals:                    4504   BIC:                        -3.136e+04
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.0139      0.000     57.643      0.000       0.013       0.014
x1             0.0028   4.82e-05     57.707      0.000       0.003       0.003
==============================================================================
Omnibus:                      195.556   Durbin-Watson:                   2.008
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              132.226
Skew:                          -0.305   Prob(JB):                     1.94e-29
Kurtosis:                       2.423   Cond. No.                         11.3
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [67]:
ans1=linregress(x=x1[:len(y_)],y=y_)

def show_regression(x_, y_, slope = 0, intercept = 0):
    plot = go.Figure()
    graph = go.Scatter(x = x_, y = y_, mode = 'markers', name = 'Points')
    graph2 = go.Scatter(x = x_, y = [intercept+slope*xi for xi in x_], name = 'Regression line')
    plot.add_traces([graph, graph2])
    
    return plot
plot = show_regression(x1[:len(y_)], y_, ans1.slope, ans1.intercept)
plot.update_layout(title='Ruble/gas dependence and regression line',xaxis_title='Gas',yaxis_title='Ruble/Dollar exchange rate')
plot.show()

Analytical test of ruble/oil dependance

In [68]:
#use regression analysis on ruble/oil prices relationship
x2_ = sm.add_constant(list(x2))
model = sm.OLS(y_,x2_[-len(y_):])
res2 = model.fit()
res2.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.048
Model:                            OLS   Adj. R-squared:                  0.048
Method:                 Least Squares   F-statistic:                     225.8
Date:                Sun, 15 May 2022   Prob (F-statistic):           7.94e-50
Time:                        13:54:09   Log-Likelihood:                 14554.
No. Observations:                4506   AIC:                        -2.910e+04
Df Residuals:                    4504   BIC:                        -2.909e+04
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.0198      0.000     43.817      0.000       0.019       0.021
x1          9.392e-05   6.25e-06     15.025      0.000    8.17e-05       0.000
==============================================================================
Omnibus:                    32878.484   Durbin-Watson:                   1.999
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              408.724
Skew:                          -0.126   Prob(JB):                     1.77e-89
Kurtosis:                       1.546   Cond. No.                         230.
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

Надо будет сделать выводы по полученным данным

In [69]:
ans2=linregress(x=x2[-len(y_):],y=y_)
def show_regression(x_, y_, slope = 0, intercept = 0):
    plot = go.Figure()
    graph = go.Scatter(x = x_, y = y_, mode = 'markers', name = 'Points')
    graph2 = go.Scatter(x = x_, y = [intercept+slope*xi for xi in x_], name = 'Regression line')
    plot.add_traces([graph, graph2])
    
    return plot
plot = show_regression(x2[-len(y_):], y_, ans2.slope, ans2.intercept)
plot.update_layout(title='Ruble/gas dependence and regression line',xaxis_title='Gas',yaxis_title='Ruble/Dollar exchange rate')
plot.show()

In [71]:
print(ans1.slope,ans2.slope)
print(ans1.slope-ans2.slope)

0.0027801277048531297 9.3924685924394e-05
0.0026862030189287356


### Conclusion
As slope of the first regression line is greater than of the second line then based on the given data, using the model of linear regression, we can make a conclusion that changes in world gas price influences ruble/dollar exchange rate more significantly than changes in world oil price.